### Attention

在之前的Seq2Seq中简单介绍了注意力机制，其通过在每个时间步中为输入序列中不同位置袋词分配不同的注意力权重。使得模型能够更加灵活地有选择地关注输入序列中的重要部分，从而更好地捕捉上下文相关性，模型的性能也会因此而提高。  

在Attention中最重要的就是如何计算得分，下图中列出了三种计算score的方法:
<img src="./images/score_function.png" alt="examples" style="zoom:30%;" />  
其中最简单的就是点积注意力，所以以点积注意力为例介绍下计算两个张量的点积注意力的步骤

#### Dot Product Attention

点积注意力顾名思义就是计算两个向量的点积

> 1. 创建两个张量: x1和x2

In [1]:
import torch
x1 = torch.randn(2, 3, 4) # (batch_size, seq_len1,  feature_dim)
x2 = torch.randn(2, 5, 4) # (batch_size, seq_len2,  feature_dim)

print(x1)
print(x2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[-0.5337, -0.4676, -0.2803, -0.6368],
         [ 0.4674, -0.6213,  1.5579,  0.2032],
         [-0.1544, -0.9061,  0.7596, -0.0978]],

        [[ 1.7761,  1.1352,  0.7950,  1.2606],
         [ 0.2405,  1.2241, -0.0210, -1.2548],
         [-0.2855, -1.1407, -1.2737, -0.9245]]])
tensor([[[-1.9028, -1.1113,  0.9790,  1.7046],
         [ 0.0688,  1.5732,  0.0434, -0.3994],
         [-1.2683,  0.6892,  0.7112,  1.4494],
         [-0.1494,  1.1914, -0.1581,  0.4710],
         [-1.2611, -0.0855, -1.4043, -0.2682]],

        [[ 0.1631, -0.7872,  0.7288,  1.4120],
         [ 0.1255, -0.1833,  0.8956, -1.5014],
         [-0.6591, -0.4399,  0.4521,  0.7559],
         [ 0.9043, -0.0518, -0.5727,  0.3202],
         [-0.3043,  0.3897, -0.0329, -0.7944]]])


> 2. 计算点积, 得到原始权重(score)  

weight中包含x1每一行和x2中每一列的点积结果, 表征了任意两组向量的其相似程度, 权重矩阵的每个元素值也就是相似度得分

<img src="./images/dot_product.png" alt="examples" style="zoom:60%;" />


In [4]:
raw_weights = torch.bmm(x1, x2.transpose(1,2))
print(raw_weights) # (2, 3, 5)

tensor([[[ 0.1752, -0.5302, -0.7677, -0.7330,  1.2773],
         [ 1.6725, -0.9587,  0.3814, -0.9606, -2.7785],
         [ 1.8777, -1.3641, -0.0301, -1.2226, -0.7683]],

        [[ 1.7554, -1.1658, -0.3576,  1.4957, -1.1257],
         [-2.7115,  1.6709, -1.6550, -0.2356,  1.4014],
         [-1.3823,  0.4206, -0.5847,  0.2342,  0.4187]]])


> 3. 对原始权重进行归一化

对权重矩阵进行归一化处理, 得到概率分布统计

In [5]:
import torch.nn.functional as F

attn_weights = F.softmax(raw_weights, dim=-1)
print(attn_weights)

tensor([[[0.1888, 0.0932, 0.0735, 0.0761, 0.5683],
         [0.6991, 0.0503, 0.1922, 0.0502, 0.0082],
         [0.7672, 0.0300, 0.1139, 0.0346, 0.0544]],

        [[0.4995, 0.0269, 0.0604, 0.3853, 0.0280],
         [0.0064, 0.5100, 0.0183, 0.0758, 0.3895],
         [0.0491, 0.2977, 0.1089, 0.2471, 0.2972]]])


> 4. 求注意力分布的加权和  

将权重矩阵与x2相乘就得到了加权和，此时的输出与x1的形状相同，可以看作是关注了x2之后新x1  

<img src="./images/attention_output.png" alt="examples" style="zoom:60%;" />

In [6]:
attn_output = torch.bmm(attn_weights, x2)
print(attn_output)

tensor([[[-1.1741,  0.0297, -0.5690,  0.2746],
         [-1.5883, -0.5060,  0.8039,  1.4716],
         [-1.6759, -0.6904,  0.7514,  1.4625]],

        [[ 0.3849, -0.4337,  0.1938,  0.8116],
         [ 0.0029,  0.0413,  0.4135, -1.0280],
         [ 0.1066, -0.0381,  0.2004, -0.4523]]])


#### Scaled Dot Production Attention
缩放点积注意力与点积注意力的主要区别是：缩放点积注意力在于计算注意力权重之前，会将点积结果也就是原始权重(得分)除以一个缩放因子，得到缩放后的原始权重，通常这个缩放因子是输入特征维度的平方根。  
之所以使用缩放因子，是因为在深度学习模型中，点积的值可能非常大，这样会引起梯度消失的情况。因此通过缩放因子可以确保softmax函数在一个较为平缓的区域内工作，减轻梯度消失的问题

In [7]:
import torch # 导入 torch
import torch.nn.functional as F # 导入 nn.functional
# 1. 创建两个张量 x1 和 x2
x1 = torch.randn(2, 3, 4) # 形状 (batch_size, seq_len1, feature_dim)
x2 = torch.randn(2, 5, 4) # 形状 (batch_size, seq_len2, feature_dim)
# 2. 计算张量点积，得到原始权重
raw_weights = torch.bmm(x1, x2.transpose(1, 2)) # 形状 (batch_size, seq_len1, seq_len2)
# 3. 将原始权重除以缩放因子
scaling_fator = x1.size(-1) ** 0.5
scaled_weights = raw_weights / scaling_fator
# 4. 归一化
attn_weights = F.softmax(raw_weights, dim=-1)
# 5. 使用注意力权重对x2加权求和
attn_output = torch.bmm(attn_weights, x2)
print(attn_output)

tensor([[[ 0.2498,  0.4867, -0.0678, -0.3209],
         [ 0.3602, -0.8344,  2.1514,  0.5412],
         [-0.0681, -0.6443,  1.7880,  0.1678]],

        [[-0.7950, -0.0455, -0.0373, -0.6455],
         [-0.3184,  0.2039, -0.2105, -0.6514],
         [ 0.2170, -1.7193, -1.1416, -1.7688]]])


#### attention中的Q、K、V
Q(查询-Query)：是指当前需要查询的信息。模型根据查询向量在输入序列中查找相关信息  
K(键-Key)：是来自输入序列的一组表示。主要用于计算注意力权重(得分)，注意力权重反映了不同位置的输入数据与查询的相关性   
V(值-Value)：也是来自输入序列的一组表示。主要是用于计算注意力的输出，其包含了在输入序列中与查询信息最相关输入信息  

<img src="./images/qkv.png" alt="examples" style="zoom:30%;" />

In [8]:
# 通过qkv的方式计算注意力
import torch
import torch.nn.functional as F
# 1.初始化三个向量
q = torch.randn(2, 3, 4)
k = torch.randn(2, 4, 4)
v = torch.randn(2, 4, 4)

# 2.计算点积
raw_weights = torch.bmm(q, k.transpose(1, 2))

# 3.缩放点积
scaled_factor = q.size(-1) ** 0.5 
scaled_weights = raw_weights / scaled_factor

# 4.归一化
attn_weights = F.softmax(raw_weights, dim=-1)

# 5.计算注意力的输出
attn_output = torch.bmm(attn_weights, v)
print(attn_output)

tensor([[[-1.2523, -0.0728, -1.3015, -0.5152],
         [-1.4100,  0.1692, -1.2541, -0.9562],
         [ 0.2499, -0.7288,  0.1072, -0.2067]],

        [[ 0.0741, -0.1362, -0.4524, -0.2669],
         [-0.6264,  0.5503, -0.1296, -0.1285],
         [-0.0802,  0.3008, -1.1604,  0.6163]]])


#### 自注意力
对自身的注意力，也就是q、k和v都是来自于同一个序列。它允许模型在统一序列中的不同位置之间建立依赖关系

In [10]:
# 通过k、q和v计算自注意力
import torch
import torch.nn as nn
import torch.nn.functional as F

# 输入序列x
x = torch.randn(2, 3, 4) # (batch_size, seq_len, fea_dim)
# 通过线性层将x转换为q k v
linear_q = nn.Linear(4, 4)
linear_k = nn.Linear(4, 4)
linear_v = nn.Linear(4, 4)

q = linear_q(x)
k = linear_k(x)
v = linear_v(x)

# 2.计算点积
raw_weights = torch.bmm(q, k.transpose(1, 2))

# 3.缩放点积
scaled_factor = q.size(-1) ** 0.5 
scaled_weights = raw_weights / scaled_factor

# 4.归一化
attn_weights = F.softmax(raw_weights, dim=-1)

# 5.计算注意力的输出
attn_output = torch.bmm(attn_weights, v)
print(attn_output.data)

tensor([[[ 0.3608, -0.1260, -0.8011,  0.5335],
         [ 0.5163, -0.0437, -0.7180,  0.3559],
         [ 0.6221, -0.0479, -1.0432,  0.6608]],

        [[-0.2247,  0.3779, -0.3741,  0.6164],
         [-0.4156,  0.1507, -0.6751,  1.1728],
         [-0.3861,  0.0892, -0.5719,  1.0286]]])


#### Multi-head Attention
多头自注意力机制是注意力机制的一种扩展，它可以使模型从不同的表示子空间中捕获输入数据的多种特征。具体而言，多头注意力在计算注意力权重和输出时，会对Q、K和V向量分别进行多次线性变换，从而获得不同的头(Head)，并进行并行计算 

<img src="./images/muitl-head.png" alt="examples" style="zoom:50%;" />

##### 多头自注意力机制的计算步骤

> 1. 初始化

设置多头自注意力的头数。每个头将处理输入数据的一个子空间

In [14]:
# 通过k、q和v计算自注意力
import torch
import torch.nn as nn
import torch.nn.functional as F

# 输入序列x
x = torch.randn(2, 3, 4) # (batch_size, seq_len, fea_dim)
# 设置头数以及每个头的维度
num_heads = 2
head_dim = 2
# 其实这里可以看出就是将
assert x.size(-1) == num_heads * head_dim

> 2. 线性变换

对QKV进行多次线性变换，每次变换使用不同的权重矩阵，这样可以得到多组不同的QKV向量

In [19]:
linear_q = nn.Linear(4, 4)
linear_k = nn.Linear(4, 4)
linear_v = nn.Linear(4, 4)

Q = linear_q(x) # (batch_size, seq_len, fea_dim)
K = linear_k(x) # (batch_size, seq_len, fea_dim)
V = linear_v(x) # (batch_size, seq_len, fea_dim)

# 将qkv分割成num_heads个头
def split_heads(tensor, num_heads):
    batch_size, seq_len, fea_dim = tensor.size()
    head_dim = fea_dim // num_heads
    # batch_size, num_heads, seq_len, head_dim) 转置操作是为了便于并行计算?
    output = tensor.view(batch_size, seq_len, num_heads, head_dim).transpose(1, 2)
    return output

Q = split_heads(Q, num_heads) # (batch_size, num_heads, seq_len, head_dim)
K = split_heads(K, num_heads) # (batch_size, num_heads, seq_len, head_dim)
V = split_heads(V, num_heads) # (batch_size, num_heads, seq_len, head_dim)

# 计算点积
raw_weights = torch.matmul(Q, K.transpose(-2, -1)) # (batch_size, num_heads, seq_len, seq_len)

> 3. 缩放点积注意力

将每组Q、K、V向量输入缩放点积注意力中进行计算，每个头将生成一个加权输出

In [24]:
# 对原始权重进行缩放
scaled_factor = Q.size(-1) ** 0.5
scaled_weights = raw_weights / scaled_factor

# 归一化
attn_weights = F.softmax(scaled_weights, dim=-1) # (batch_size, num_heads, seq_len, seq_len)

# 计算加权和
attn_output = torch.matmul(attn_weights, V) # (batch_size, num_heads, seq_len, head_dim)

> 4. 合并

将所有头的加权输出拼接起来，并进行一次线性变换，得到多头自注意力的最终输出

In [30]:
def combine_heads(tensor, num_heads):
    batch_size, num_heads, seq_len, head_dim = tensor.size()
    output = tensor.transpose(1,2).contiguous().view(batch_size, seq_len, num_heads * head_dim)
    return output

attn_outputs = combine_heads(attn_output, num_heads) # (batch_size, seq_len, fea_dim)
linear = nn.Linear(4, 4)
attn_outputs = linear(attn_outputs)
print(attn_outputs.data)

tensor([[[ 0.9651, -0.2872,  0.4699,  0.8848],
         [ 1.0233, -0.3343,  0.4772,  0.8773],
         [ 1.0863, -0.3252,  0.5337,  0.9121]],

        [[ 1.0693, -0.2549,  0.5669,  0.8887],
         [ 1.0825, -0.2615,  0.5737,  0.8923],
         [ 1.0932, -0.2530,  0.5884,  0.8983]]])
